In [1]:
import math
import numpy as np
import pandas as pd
from tvDatafeed import TvDatafeed,Interval
import credentials_tv
from datetime import date, timedelta, datetime
import time
import talib as ta

In [2]:
def get_fno_stocklist():
  df = pd.read_csv('FNO_LIST.csv')
  return df

In [3]:
def download_stock(tckr,interval, n_bars):
  stock_data = tv.get_hist(tckr,'NSE',interval,n_bars=n_bars,fut_contract=1)
  if(len(stock_data)<1):
    stock_data = pd.DataFrame()
  stock_data.drop(stock_data[['symbol']], inplace = True, axis=1)
  stock_data.index = stock_data.index.strftime("%Y-%m-%d")
  return stock_data

In [4]:
# get credentials for tradingview
username = credentials_tv.username
password = credentials_tv.password

In [5]:
# initialize tradingview
tv=TvDatafeed(auto_login=False)




You need to login manually

 Press 'enter' to open the browser 
opening browser. Press enter once lgged in return back and press 'enter'. 

DO NOT CLOSE THE BROWSER


Message: invalid argument: user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir

you are using nologin method, data you access may be limited


In [6]:
# # initialize tradingview
#tv=TvDatafeed(username=username,password=password)

In [7]:
download_stock('RELIANCE', Interval.in_daily, n_bars=1)

,open,high,low,close,volume
datetime,,,,,
2022-04-25,2745.1,2757.0,2661.0,2692.2,13627250.0


In [8]:
def Gann_prediction(LTP):
  
  # find sq root of the ltp
  sq_rt_ltp = math.sqrt(LTP)
  #ceil the sq rt
  cieled_sq_rt = math.ceil(sq_rt_ltp)
  #get starting number for iteration
  lowest_num = cieled_sq_rt-2
  block = [lowest_num, lowest_num+1, lowest_num+2]
  
  for i in (block):
    centre_1 = math.pow(i+0.125, 2)
    centre_2 = math.pow(i+0.125*2, 2)
    centre_3 = math.pow(i+0.125*3, 2)
    centre_4 = math.pow(i+0.125*4, 2)
    centre_5 = math.pow(i+0.125*5, 2)
    centre_6 = math.pow(i+0.125*6, 2)
    centre_7 = math.pow(i+0.125*7, 2)
    centre_8 = math.pow(i+0.125*8, 2)
    if(i==block[0]):
      level_1 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
    if(i==block[1]):
      level_2 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
    if(i==block[2]):
      level_3 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
  
  #merge list
  Gann_list = level_1 + level_2 + level_3


  insrt_loc = 0  
  for j in range(len(Gann_list)):
    if(Gann_list[j] < LTP < Gann_list[j+1]):
      insrt_loc = j+1
      Gann_list.insert(insrt_loc, LTP)
      break

  Resistance_1, Resistance_2, Resistance_3, Resistance_4 = (Gann_list[insrt_loc+2], Gann_list[insrt_loc+3],
                                                                       Gann_list[insrt_loc+4], Gann_list[insrt_loc+5])

  Support_1, Support_2, Support_3, Support_4 = (Gann_list[insrt_loc-2], Gann_list[insrt_loc-3],
                                                                       Gann_list[insrt_loc-4], Gann_list[insrt_loc-5])

  buy_above = Gann_list[insrt_loc+1]
  c1 = 0.99995
  sell_target_1, sell_target_2, sell_target_3, sell_target_4 =  (round(c1*Resistance_1,2), round(c1*Resistance_2,2), 
                                                                              round(1*Resistance_3,2), round(c1*Resistance_4,2))
  sl_long = Gann_list[insrt_loc-1]
  sell_below = Gann_list[insrt_loc-1]
  c2 = 1.00005
  buy_target_1, buy_target_2, buy_target_3, buy_target_4 = (round(c2*Support_1,2), round(c2*Support_2,2), round(c2*Support_3,2), 
                                                                        round(c2*Support_4,2))
  sl_short = Gann_list[insrt_loc+1]
  
  return (buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
          sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4)
  

In [9]:
def profit_percentage(prediction_list_df): # this function will use multiple levls of filtering
    
    stock_tckr = prediction_list_df['stock_tckr']
    buy_price =  prediction_list_df['buy_above'] #buying price for long trade
    sell_price = prediction_list_df['sell_below'] # selling price for short trade
    sl_long = prediction_list_df['sl_long']
    sl_short = prediction_list_df['sl_short']

    # Percentage profit with target 1 achieved for long trade
    pct_profit_long_tgt_1 = (prediction_list_df['sell_target_1'] - buy_price) * 100/ buy_price
    # Percentage profit with target 2 achieved for long trade
    pct_profit_long_tgt_2 = (prediction_list_df['sell_target_2'] - buy_price) * 100/ buy_price
    # Percentage profit with target 3 achieved for long trade
    pct_profit_long_tgt_3 = (prediction_list_df['sell_target_3'] - buy_price) * 100/ buy_price
    # Percentage profit with target 4 achieved for long trade
    pct_profit_long_tgt_4 = (prediction_list_df['sell_target_4'] - buy_price) * 100/ buy_price

    # Percentage profit with target 1 achieved for short trade
    pct_profit_short_tgt_1 = (sell_price - prediction_list_df['buy_target_1']) *100/ prediction_list_df['buy_target_1']
    # Percentage profit with target 2 achieved for short trade
    pct_profit_short_tgt_2 = (sell_price - prediction_list_df['buy_target_2']) *100/ prediction_list_df['buy_target_2']
    # Percentage profit with target 3 achieved for short trade
    pct_profit_short_tgt_3 = (sell_price - prediction_list_df['buy_target_3']) *100/ prediction_list_df['buy_target_3']
    # Percentage profit with target 4 achieved for short trade
    pct_profit_short_tgt_4 = (sell_price - prediction_list_df['buy_target_4']) *100/ prediction_list_df['buy_target_4']
    
    # check stop loss vs pct profit
    # Risk reward percentage: long trade vs stop loss
    loss_pct_against_long_trade = (buy_price - sl_long) * 100/ buy_price
    # Risk reward percentage: short trade vs stop loss
    loss_pct_against_short_trade = (sl_short - sell_price) * 100/ sl_short

    return (stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
            pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
            loss_pct_against_long_trade, loss_pct_against_short_trade)

In [10]:
def stock_filtering(long_profit, long_loss, short_profit, short_loss):
    if((long_profit>(2*long_loss) or short_profit>(2*short_loss)) and (long_profit>2 or short_profit>2)):
        return "good_stock"
    else:
        return "remove_stock"


In [11]:
# Main 
fno_stock_list = get_fno_stocklist()
prediction_list =[]
risk_rewards = []

# start_date = pd.Timestamp('today') - timedelta(days=1)# previous 1 day from today
# start_date = start_date.strftime("%Y-%m-%d")

# current_date = pd.Timestamp('today') + timedelta(days=1) # this adjustment is required because python does not include last date of the range.
# current_date = current_date.strftime("%Y-%m-%d")


In [12]:
# Gann prediction algorithm
interval = Interval.in_daily
n_bars = 1
for i in range(len(fno_stock_list)):
#for i in range(0,5): # test with 3 stocks
  fno_stock_tckr = fno_stock_list['symbol'][i] # get ticker symbol
  print(fno_stock_tckr)
  fno_stock_data = download_stock(fno_stock_tckr, interval ,n_bars) # download data from yfinance 
  
  if(len(fno_stock_data)>0 and not(pd.isnull(fno_stock_data.iloc[0,3]))):
    current_day_close = round(fno_stock_data.values[-1][3],2)

    (buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
           sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4)= Gann_prediction(current_day_close)
          
    prediction_list.append([fno_stock_tckr,buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
           sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4])



prediction_list_df = pd.DataFrame(prediction_list, columns=['stock_tckr','buy_above', 'sl_long', 'sell_target_1', 'sell_target_2', 
        'sell_target_3', 'sell_target_4', 'sell_below', 'sl_short', 'buy_target_1', 'buy_target_2', 'buy_target_3', 'buy_target_4'])



BANDHANBNK
AUBANK
MOTHERSUMI
TATACONSUM
DABUR
IBULHSGFIN
ATUL
BERGEPAINT
POLYCAB
SRF
ASHOKLEY
GODREJCP
CIPLA
SHREECEM
PEL
NAVINFLUOR
VOLTAS
JKCEMENT
AMBUJACEM
ZYDUSLIFE
MARUTI
INDIACEM
UPL
BALKRISIND
BRITANNIA
ABBOTINDIA
TORNTPHARM
MARICO
COROMANDEL
PIDILITIND
PFIZER
ABB
NESTLEIND
TVSMOTOR
BHARATFORG
ABCAPITAL
TITAN
ASIANPAINT
CHOLAFIN
CUMMINSIND
IRCTC
APLLTD
ESCORTS
DALBHARAT
ACC
BHARTIARTL
TATAMOTORS
SBICARD
SBILIFE
BAJAJFINSV
JUBLFOOD
RAMCOCEM
SBIN
HONAUT
SUNPHARMA
GUJGASLTD
LALPATHLAB
ICICIBANK
LT
COLPAL
BAJFINANCE
APOLLOTYRE
GLENMARK
DIVISLAB
BIOCON
ULTRACEMCO
ITC
BATAINDIA
EICHERMOT
METROPOLIS
INDIGO
TRENT
MRF
HAL
GRANULES
EXIDEIND
ALKEM
SYNGENE
WHIRLPOOL
ASTRAL
IEX
DEEPAKNTR
PFC
GRASIM
HAVELLS
NTPC
MANAPPURAM
CUB
AMARAJABAT
BOSCHLTD
UBL
MFSL
DLF
HDFCAMC
LUPIN
ABFRL
ICICIPRULI
HINDUNILVR
HEROMOTOCO
TCS
HDFC
CONCOR
CANBK
BANKBARODA
INDUSINDBK
HDFCBANK
DIXON
AARTIIND
ADANIPORTS
MUTHOOTFIN
PIIND
IOC
RECLTD
PETRONET
TATACHEM
GODREJPROP
TORNTPOWER
INDHOTEL
PAGEIND
INDIAMART
DRREDDY
HD

In [13]:
risk_rewards =[]

In [14]:
#filtering low profit stocks
#for i in range(0,100): # test with 3 stocks
for i in range(len(prediction_list_df)):
  (stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
  pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
  loss_pct_against_long_trade, loss_pct_against_short_trade) = profit_percentage(prediction_list_df.iloc[i])

  risk_rewards.append([stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
  pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
  loss_pct_against_long_trade, loss_pct_against_short_trade])

risk_rewards_df = pd.DataFrame(risk_rewards, columns = ['stock_tckr', 'pct_profit_long_tgt_1', 'pct_profit_long_tgt_2', 'pct_profit_long_tgt_3',
 'pct_profit_long_tgt_4', 'pct_profit_short_tgt_1', 'pct_profit_short_tgt_2', 'pct_profit_short_tgt_3', 'pct_profit_short_tgt_4', 
 'loss_pct_against_long_trade', 'loss_pct_against_short_trade'])

# Merge both data frames, Gann predictions and risk rewards for evaluation
reduced_stocks_df = pd.merge(prediction_list_df, risk_rewards_df,  on = 'stock_tckr')

# Filter elimination criteria
reduced_stocks_df['stock_filter'] = reduced_stocks_df.apply(lambda x: stock_filtering(x.pct_profit_long_tgt_2, x.loss_pct_against_long_trade, x.pct_profit_short_tgt_2,
                                                        x.loss_pct_against_short_trade), axis=1)
reduced_stocks_df = reduced_stocks_df[reduced_stocks_df['stock_filter']!='remove_stock'] # drop low profit stocks
reduced_stocks_df=reduced_stocks_df.reset_index(drop=True) #reset index


EMA strategy to get direction

In [15]:
# start_date = pd.Timestamp('today') - timedelta(days=30) # previous 1 day from today
# start_date = start_date.strftime("%Y-%m-%d")


In [16]:
# Intraday trades
signal_list =[]
interval = Interval.in_30_minute
nbars=13
for i in range(len(reduced_stocks_df)):
  #for i in range(0,10):
  fno_stock_tckr = reduced_stocks_df['stock_tckr'][i] # get ticker symbol
  data = download_stock(fno_stock_tckr, interval ,nbars) # download data from yfinance 
  if(len(data)>0):
    #calculate EMA and decide the direction
    ema = ta.EMA(data.close, timeperiod=13)
    
    #long_ema = ta.EMA(data['Close'], timeperiod=50)
    #first 15 min candle of the day
    previous_high = data.high[-2]
    latest_high = data.high[-1]
    previous_low = data.low[-2]
    latest_low = data.low[-1]
    previous_close = data.close[-2]
    latest_close = data.close[-1]


    if((previous_close > ema[-1]) and (latest_close>previous_high)):
      signal ='sell'
    if((previous_close < ema[-1]) and (latest_close<previous_low)):
      signal = 'buy'
    else:
      signal = 'none'
    
    signal_list.append([fno_stock_tckr, signal])

#create df of signal direction
signal_list_df = pd.DataFrame(signal_list, columns =['stock_tckr', 'signal'])

In [ ]:
# create a df to backtest using actual data against predicted data
final_stocks_for_tomorrow = pd.merge(reduced_stocks_df,signal_list_df, on = 'stock_tckr')

In [ ]:
final_stocks_for_tomorrow.rename(columns={'stock_tckr': 'Stock symbol', 'buy_above': 'Buy above', 
'sl_long': 'Stop loss long', 'sell_target_1':'Sell target 1','sell_target_2':'Sell target 2','sell_target_3':'Sell target 3',
'sell_target_4':'Sell target 4', 'sell_below':'Sell below', 'sl_short':'Stop loss short', 'buy_target_1':'Buy target 1', 'buy_target_2':'Buy target 2',
'buy_target_3':'Buy target 3', 'buy_target_4':'Buy target 4',
'pct_profit_long_tgt_4':'Max Gain(%) long', 'pct_profit_short_tgt_4':'Max Gain(%) short',
'signal':'Action'}, inplace=True)

In [ ]:
final_stocks_for_tomorrow.drop(['pct_profit_long_tgt_1', 'pct_profit_long_tgt_2', 'pct_profit_long_tgt_3','pct_profit_short_tgt_1', 
'pct_profit_short_tgt_2','pct_profit_short_tgt_3', 'loss_pct_against_long_trade', 'loss_pct_against_short_trade','stock_filter'], 
       axis=1, inplace=True)

In [ ]:
tomorrow=date.today() + timedelta(days=1)
final_stocks_for_tomorrow.insert(loc = 0,
          column = 'Date',
          value = tomorrow)
final_stocks_for_tomorrow = final_stocks_for_tomorrow[final_stocks_for_tomorrow.Action != 'none']

In [ ]:
final_stocks_for_tomorrow.to_csv('Predictions/fno_predictions_for_tomorrow.csv')